# Realización del Modelo

In [2]:
from funciones import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import xgboost as xgb
from xgboost import XGBRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor





import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.SettingWithCopyWarning)

In [ ]:
# Como primer paso se creara un modelo que prediga el precio de la electricidad del día siguiente
# A partir del dataset original se harán 3 diferentes preprocesamientos y se elegirá al que mejor métrica de
# Como el precio del día sugiente es la variable objetivo, se tendrá que hacer uso de la funcion shift(-1)

In [19]:
df=juntar_todo()

In [22]:
def prepro_1_ML(df):
    ''' Elige ciertas columnas del dataset original, mete las variables temporales y lo que sería la columna del precio del día siguiente, es decir, 
        la varia objetivo o la 'y'.
    '''
    df=df[['Fecha','Generación renovable','Generación no renovable','Saldo I. internacionales','Demanda',
    'T_Max_España','T_Min_España','T_Media_España','Precio_gas','Precio_PT','precio_FR','precio_ES']]
    df=crea_var_temporales(df)
    df['y']=df.precio_ES.shift(-1)
    df=df.dropna()
    return df

In [ ]:
# Crearé una función que calcule las métricas de 5 modelos diferentes

In [25]:
def modelos(df):
    ''' Una función que dado un dataframe calcula las metricas de 5 modelos de ML 
    '''
    X = df.drop(['Fecha','y'], axis=1)
    y = df['y']
    scaler = MinMaxScaler()
    X=scaler.fit_transform(X)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
    model1 = LinearRegression()
    model2 = RandomForestRegressor()
    model3 =XGBRegressor()
    model4 = KNeighborsRegressor()
    model5 =GradientBoostingRegressor()
    
    model1.fit(X_train, y_train)
    y_pred = model1.predict(X_test)
    y_pred_train= model1.predict(X_train)
    
    rmse_train1 = np.sqrt(mean_squared_error(y_train, y_pred_train))
    r2_train1 = r2_score(y_train, y_pred_train)    
        
    rmse1 = np.sqrt(mean_squared_error(y_test, y_pred))
    r2_1 = r2_score(y_test, y_pred)
    
    
    model2.fit(X_train, y_train)  
    y_pred = model2.predict(X_test)
    y_pred_train= model2.predict(X_train)
    
    rmse_train2 = np.sqrt(mean_squared_error(y_train, y_pred_train))
    r2_train2 = r2_score(y_train, y_pred_train)    
        
    rmse2 = np.sqrt(mean_squared_error(y_test, y_pred))
    r2_2 = r2_score(y_test, y_pred)    
    
    model3.fit(X_train, y_train)  
    y_pred = model3.predict(X_test)
    y_pred_train= model3.predict(X_train)
    
    rmse_train3 = np.sqrt(mean_squared_error(y_train, y_pred_train))
    r2_train3 = r2_score(y_train, y_pred_train)    
        
    rmse3 = np.sqrt(mean_squared_error(y_test, y_pred))
    r2_3 = r2_score(y_test, y_pred)

    model4.fit(X_train, y_train)  
    y_pred = model4.predict(X_test)
    y_pred_train= model4.predict(X_train)
    
    rmse_train4 = np.sqrt(mean_squared_error(y_train, y_pred_train))
    r2_train4 = r2_score(y_train, y_pred_train)    
        
    rmse4 = np.sqrt(mean_squared_error(y_test, y_pred))
    r2_4 = r2_score(y_test, y_pred)    
    
    model5.fit(X_train, y_train)  
    y_pred = model5.predict(X_test)
    y_pred_train= model5.predict(X_train)
    
    rmse_train5 = np.sqrt(mean_squared_error(y_train, y_pred_train))
    r2_train5 = r2_score(y_train, y_pred_train)    
        
    rmse5 = np.sqrt(mean_squared_error(y_test, y_pred))
    r2_5 = r2_score(y_test, y_pred)
    
    results = {'Model': ['Linear Regression','Random Forest','XGBoost','K-Nearest Neighbors','Gradient Boosting'],'RMSE train': [rmse_train1,rmse_train2,rmse_train3,rmse_train4,rmse_train5],'R2 train': [r2_train1,r2_train2,r2_train3,r2_train4,r2_train5], 'RMSE test':[rmse1,rmse2,rmse3,rmse4,rmse5],'R2 test': [r2_1,r2_2,r2_3,r2_4,r2_5] }

    return pd.DataFrame(results)

In [27]:
modelos(prepro_1_ML(df))

,Model,RMSE train,R2 train,RMSE test,R2 test
0,Linear Regression,15.132607,0.926291,13.842908,0.915364
1,Random Forest,6.215377,0.987566,14.090170,0.912313
2,XGBoost,1.054143,0.999642,14.645828,0.905261
3,K-Nearest Neighbors,13.680675,0.939757,15.108336,0.899183
4,Gradient Boosting,9.819797,0.968962,13.157232,0.923541


In [32]:
def prepro_2_ML(df):
    ''' Elige ciertas columnas del dataset original, mete las variables temporales y lo que sería la columna del precio del día siguiente, es decir, 
        la varia objetivo o la 'y'.
    '''
    df_1=df[['Fecha','Saldo I. internacionales','Demanda',
    'T_Max_España','T_Min_España','T_Media_España','Precio_gas','Precio_PT','precio_FR','precio_ES']]
    df_1['Generación']=df['Generación renovable']+df['Generación no renovable']
    df_1['Bombeo']=df_1['Generación']-df['Demanda']-df['Saldo I. internacionales']
    df_1=crea_var_temporales(df_1)
    df_1['y']=df_1.precio_ES.shift(-1)
    df_1=df_1.dropna()
    return df_1
modelos(prepro_2_ML(df))

,Model,RMSE train,R2 train,RMSE test,R2 test
0,Linear Regression,15.145386,0.926167,13.878890,0.914923
1,Random Forest,5.986571,0.988464,14.234880,0.910503
2,XGBoost,1.092448,0.999616,13.838900,0.915413
3,K-Nearest Neighbors,13.799204,0.938708,14.569027,0.906252
4,Gradient Boosting,9.811964,0.969011,13.347857,0.921309


In [33]:
def prepro_3_ML(df):
    ''' Elige ciertas columnas del dataset original, mete las variables temporales y lo que sería la columna del precio del día siguiente, es decir, 
        la varia objetivo o la 'y'. Esta vez le añado una columna que dice si es fin de semana mañana
    '''
    df_1=df[['Fecha','Saldo I. internacionales','Demanda',
    'T_Media_España','Precio_gas','precio_FR','precio_ES']]
    df_1['Generación']=df['Generación renovable']+df['Generación no renovable']
    df_1['Bombeo']=df_1['Generación']-df['Demanda']-df['Saldo I. internacionales']
    df_1['Finde_mañana']=(df_1.index % 7 == 0) | ((df_1.index -1) % 7==0)
    df_1=crea_var_temporales(df_1)
    df_1['y']=df_1.precio_ES.shift(-1)
    df_1=df_1.dropna()
    return df_1    
modelos(prepro_3_ML(df))    

,Model,RMSE train,R2 train,RMSE test,R2 test
0,Linear Regression,14.714195,0.930311,13.344293,0.921351
1,Random Forest,6.014983,0.988354,13.577770,0.918575
2,XGBoost,0.992052,0.999683,13.758149,0.916397
3,K-Nearest Neighbors,13.939543,0.937455,15.042535,0.900059
4,Gradient Boosting,9.373383,0.971720,12.982144,0.925562


In [ ]:
# De estos 3 preprocesamientos el número 3 es el que da los mejores resultados, por lo que será el dataset que se utilzará a futuro
# El objetivo principal no es hacer modelos para calcular el día siguiente, por lo que se necsita hacer uso de modelos de serie de tiempo